In [0]:
!git clone https://github.com/soumyac1999/Music-Translation.git

In [0]:
!mv Music-Translation/Data Data/

In [0]:
pip install --upgrade soundfile librosa

In [0]:
import numpy as np
import librosa
import soundfile as sf
import shutil, os
# import argparse

# parser = argparse.ArgumentParser()

# parser.add_argument('--src', help="Src root", required=True)
# parser.add_argument('--dst', help="Dest root", required=True)

if __name__ == '__main__':
	# args = parser.parse_args()
	src_root = 'Data/'
	dest_root = 'Preprocessed/'
	for instrument_name in os.listdir(src_root): # instrument wise
		print(instrument_name)
		for data_split in os.listdir(os.path.join(src_root, instrument_name)): # split dir
			for filename in os.listdir(os.path.join(src_root, instrument_name, data_split)):
				# do  something about it -> process
				# now load and save npy files
				srcpath = os.path.join(src_root, instrument_name, data_split, filename)
				y, sr = librosa.load(srcpath, sr=32000)
				S = librosa.feature.mfcc(y, sr=sr)
				dstpath = os.path.join(dest_root, instrument_name, data_split, filename.split('.')[0])
				os.makedirs(os.path.join(dest_root, instrument_name, data_split), exist_ok=True)
				np.save(dstpath, S)

In [0]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import random

# y, sr = librosa.load('2342.wav', duration=5, sr=16000)
# # sf.write('2330_0.wav', y, sr)
# # ipd.Audio('2330_0.wav')
# D = np.abs(librosa.stft(y))**2
# S = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=64, fmax=8000)
# plt.subplot(121)
# S_dB = librosa.power_to_db(S, ref=np.max)
# librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Mel-frequency spectrogram')
# plt.tight_layout()
# # plt.show()

# y_hat = librosa.feature.inverse.mel_to_audio(S, sr)
# # sf.write('2330_1.wav', y_hat, sr)
# # ipd.Audio('2330_1.wav')
# ipd.Audio(y_hat, rate=sr)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

device = 'cuda:0'
class InstrumentDataset(object):
	def __init__(self, datadir, normalize=False):
		super(InstrumentDataset, self).__init__()
		self.datadir = datadir
		self.files = os.listdir(datadir)
		self.len = len(self.files)
		self.normalize = normalize
		self.ylen = 157

	def __len__(self):
		return self.len

	def __getitem__(self, idx):
		S = np.load(os.path.join(self.datadir, self.files[idx]))
		# y, sr = librosa.load(os.path.join(self.datadir, self.files[idx]), duration=5, sr=16000)
		# D = np.abs(librosa.stft(y))**2
		# S = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=64, fmax=8000)
		x = torch.tensor(S.T, dtype=torch.float)
		return x

class Encoder(nn.Module):
    def __init__(self, input_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout):
        super(Encoder,self).__init__()
        self.input_dim = input_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(input_dim, enc_hid_dim, n_layers, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        # self.dropout = lambda x: x
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = src
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        # print("Values returned by encoder: ", outputs.shape, hidden.shape)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)

        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return torch.nn.functional.softmax(attention, dim=1)
        

class Decoder(nn.Module):
    def __init__(self, output_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
                
        self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim, dec_hid_dim, n_layers)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + output_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        self.dropout = lambda x: x
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        # print("input.size in Decoder forward", input.size())
        #input = [1, batch size]
        
        embedded = input
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)

         #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        # print("Output.size() in Decoder forward: ", output.size())
        return output, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, n_classes, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.DCNet = nn.Sequential(nn.Linear(encoder.enc_hid_dim*2,n_classes))
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        # print("max_len: ", max_len)
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        predictions = self.DCNet(encoder_outputs[-1,:,:])
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        # print("Input.size in Seq2Seq forward: ", input.size())
        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # print("Teacher forcing on? ", teacher_force)
            #get the highest predicted token from our predictions
            # top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else output
            # print("Input shape used for next pass to decoder: ", input.shape)

        return outputs,predictions

INPUT_DIM = 20
OUTPUT_DIM = 20
ENC_HID_DIM = 32
DEC_HID_DIM = 32
ENC_N_LAYERS = 4
DEC_N_LAYERS = 1
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
N_CLASSES = 2
datasetPaths = ['Preprocessed/Piano/train/','Preprocessed/Tabla/train/']
assert N_CLASSES == len(datasetPaths)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_N_LAYERS, ENC_DROPOUT)
decs = [Decoder(OUTPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_N_LAYERS, DEC_DROPOUT, attn) for f in range(N_CLASSES)]

models = [Seq2Seq(enc, dec, N_CLASSES, device).to(device) for dec in decs]

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
optimizers = [optim.Adam(model.parameters()) for model in models]
pretrained = False
pretrained_path = ''
if(pretrained):
  for i,model in enumerate(models):
    checkpoint = torch.load(pretrained_path + '/content/drive/My Drive/encde/model%d.pth'%(i))
    print(checkpoint.keys())
    model.load_state_dict(checkpoint['model_state_dict'])
else:
  for model in models:            
    model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Each model has {count_parameters(model):,} trainable parameters')

criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()

trainsets = [InstrumentDataset(f, True) for f in datasetPaths]
trainloaders = [DataLoader(x, batch_size=75, shuffle=True, num_workers=8) for x in trainsets]

for model in models:
  model.train()
for epoch in range(60):
  for f in range(N_CLASSES):
    for i,x in enumerate(trainloaders[f]):
      x = x.to(device).transpose(0,1)
      src = x
      tgt = x
      predictions_gt = torch.tensor(np.ones(x.shape[1])*f,dtype=torch.long).to(device)
      optimizers[f].zero_grad()
      output,predictions = models[f](src, tgt)
      # predictions is of size (batch_size, n_classes)   
      # print("Should be same", output.shape, tgt.shape)

      loss1 = criterion1(output, tgt)
      loss2 = criterion2(predictions,predictions_gt)
      (loss1-loss2).backward(retain_graph=True)
      loss2.backward()
      # torch.nn.utils.clip_grad_norm_(model.parameters(), 100)
      optimizers[f].step()
      # print(torch.abs(x[1]-output[1]))
      print('epoch: %d'%(epoch), 'iter: %d'%(i), 'class: %d'%(f), 'loss1: %f'%(loss1.item()),'loss2: %f'%(loss2.item()))
  if(epoch%5 == 0):
    for i,model in enumerate(models):
      torch.save({
                  'epoch': epoch,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizers[i].state_dict()
                  },'/content/drive/My Drive/encde/model%d.pth'%(i))

# model.eval()
# with torch.no_grad():
#     output = model(x, x, 0) #turn off teacher forcing
#     loss = criterion(x, x)

# plt.subplot(122)
# output = output.detach().cpu()*sig + mu
# S_dB = librosa.power_to_db(output[:,0,:].T, ref=np.max)
# librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Mel-frequency spectrogram')
# plt.tight_layout()
# plt.show()

# y_hat = librosa.feature.inverse.mel_to_audio(output.numpy()[:,0,:].T, sr)
# sf.write('2330_2.wav', y_hat, sr)
# # ipd.Audio('2330_2.wav')
# ipd.Audio(y_hat, rate=sr)

In [0]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import random

# y, sr = librosa.load('2342.wav', duration=5, sr=16000)
# # sf.write('2330_0.wav', y, sr)
# # ipd.Audio('2330_0.wav')
# D = np.abs(librosa.stft(y))**2
# S = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=64, fmax=8000)
# plt.subplot(121)
# S_dB = librosa.power_to_db(S, ref=np.max)
# librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Mel-frequency spectrogram')
# plt.tight_layout()
# # plt.show()

# y_hat = librosa.feature.inverse.mel_to_audio(S, sr)
# # sf.write('2330_1.wav', y_hat, sr)
# # ipd.Audio('2330_1.wav')
# ipd.Audio(y_hat, rate=sr)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

device = 'cuda:0'
class InstrumentDataset(object):
	def __init__(self, datadir, normalize=False):
		super(InstrumentDataset, self).__init__()
		self.datadir = datadir
		self.files = os.listdir(datadir)
		self.len = len(self.files)
		self.normalize = normalize
		self.ylen = 157

	def __len__(self):
		return self.len

	def __getitem__(self, idx):
		S = np.load(os.path.join(self.datadir, self.files[idx]))
		# y, sr = librosa.load(os.path.join(self.datadir, self.files[idx]), duration=5, sr=16000)
		# D = np.abs(librosa.stft(y))**2
		# S = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=64, fmax=8000)
		x = torch.tensor(S.T, dtype=torch.float)
		if self.normalize:
			mu = x.mean(axis=0, keepdims=True)
			sig = x.std(axis=0, keepdims=True)
			x = (x-mu)/sig
		return x

class Encoder(nn.Module):
    def __init__(self, input_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout):
        super(Encoder,self).__init__()
        self.input_dim = input_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(input_dim, enc_hid_dim, n_layers, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        # self.dropout = lambda x: x
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = src
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        # print("Values returned by encoder: ", outputs.shape, hidden.shape)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)

        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return torch.nn.functional.softmax(attention, dim=1)
        

class Decoder(nn.Module):
    def __init__(self, output_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
                
        self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim, dec_hid_dim, n_layers)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + output_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        self.dropout = lambda x: x
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        # print("input.size in Decoder forward", input.size())
        #input = [1, batch size]
        
        embedded = input
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)

         #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        #output = [bsz, output dim]
        # print("Output.size() in Decoder forward: ", output.size())
        return output, hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, n_classes, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.DCNet = nn.Sequential(nn.Linear(encoder.enc_hid_dim*2,n_classes))
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        # print("max_len: ", max_len)
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        predictions = self.DCNet(encoder_outputs[-1,:,:])
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        # print("Input.size in Seq2Seq forward: ", input.size())
        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # print("Teacher forcing on? ", teacher_force)
            #get the highest predicted token from our predictions
            # top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else output
            # print("Input shape used for next pass to decoder: ", input.shape)

        return outputs,predictions

INPUT_DIM = 20
OUTPUT_DIM = 20
ENC_HID_DIM = 32
DEC_HID_DIM = 32
ENC_N_LAYERS = 4
DEC_N_LAYERS = 1
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
N_CLASSES = 2
datasetPaths = ['Preprocessed/Piano/train/','Preprocessed/Tabla/train/']
assert N_CLASSES == len(datasetPaths)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_N_LAYERS, ENC_DROPOUT)
decs = [Decoder(OUTPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_N_LAYERS, DEC_DROPOUT, attn) for f in range(N_CLASSES)]

models = [Seq2Seq(enc, dec, N_CLASSES, device).to(device) for dec in decs]

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
optimizers = [optim.Adam(model.parameters()) for model in models]
pretrained = True
pretrained_path = '.'
if(pretrained):
  for i,model in enumerate(models):
    checkpoint = torch.load(pretrained_path + '/model%d.pth'%(i))
    print(checkpoint.keys())
    model.load_state_dict(checkpoint['model_state_dict'])
else:
  for model in models:            
    model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Each model has {count_parameters(model):,} trainable parameters')

criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()

testsets = [InstrumentDataset('Preprocessed/Piano/test/'), InstrumentDataset('Preprocessed/Tabla/test/')]
testloaders = [DataLoader(x, batch_size=75, shuffle=False, num_workers=8) for x in testsets]

model1 = models[0]
model2 = models[1]

model1.eval()
model2.eval()
# os.makedirs('results/piano', exist_ok=True)
with torch.no_grad():
  for i,x in enumerate(testloaders[1]):
    x = x.transpose(0,1)
    x = x.to(device)
    output = model1(x, x, 0)[0].transpose(0,1)
    x = x.transpose(0,1).detach().cpu().numpy()
    # x and output are batch_size x seq_len x filterbanks
    print(output.shape, sig.shape, mu.shape, x.shape)
    sig = sig.detach().cpu().numpy()
    mu = mu.detach().cpu().numpy()
    output = output.detach().cpu().numpy()
    x = x
    for j in range(output.shape[0]):
      y_hat = librosa.feature.inverse.mfcc_to_audio(output[j,:,:].T)
      sf.write('/content/drive/My Drive/results/tabla/tabla2piano_'+str(i*output.shape[1]+j)+'.wav', y_hat, 32000)
      print('Saved: /content/drive/My Drive/results/tabla/tabla2piano_'+str(i*output.shape[1]+j)+'.wav')
      y_hat = librosa.feature.inverse.mfcc_to_audio(x[j,:,:].T)
      sf.write('/content/drive/My Drive/results/tabla/tabla_'+str(i*output.shape[1]+j)+'.wav', y_hat, 32000)
      print('Saved: /content/drive/My Drive/results/tabla/tabla_'+str(i*output.shape[1]+j)+'.wav')

In [0]:
from google.colab import drive
drive.mount('/content/drive')